In [ ]:
#Imports
import pandas as pd
import re
from datetime import datetime, timedelta
import numpy as np

In [ ]:
# Connect to google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Read the csv file for Real Estate Data
df = pd.read_csv('/content/drive/MyDrive/DWDM Data/RealEstateData.csv')
df

,Url,Title,Type,Price,Location,Baths,Beds,Area,Purpose,Date_Added,Description
0,https://www.zameen.com//Property/islamabad_tar...,10 Kanal Empty Land Available For Rent,Commercial Plot,4 Lakh,"Tarnol, Islamabad, Islamabad Capital",-,-,10 Kanal,For Rent,3 days ago,10 kanal empty land available for rent in Isla...
1,https://www.zameen.com//Property/islamabad_gt_...,Fair-priced 27000 Square Feet Commercial Plot ...,Commercial Plot,15 Lakh,"GT Road, Islamabad, Islamabad Capital",-,-,6 Kanal,For Rent,2 weeks ago,6 Kanal Land On main GT Road For Rent Ideal Pl...
2,https://www.zameen.com//Property/islamabad_isl...,20 Kanal Land On Main Isb Highway Available Fo...,Commercial Plot,20 Lakh,"Islamabad Expressway, Islamabad, Islamabad Cap...",-,-,20 Kanal,For Rent,2 weeks ago,20 Kanal Land on Main ISB Highway Available Fo...
3,https://www.zameen.com//Property/islamabad_g_1...,Industrial Building Available For Rent,Industrial Land,55 Thousand,"G-12, Islamabad, Islamabad Capital",-,-,1 Kanal,For Rent,2 days ago,A Industrial Plot Located In G12 Just 1km Away...
4,https://www.zameen.com//Property/islamabad_mir...,Industrial Building Available For Rent,Industrial Land,50 Thousand,"Mira Abadi, Islamabad, Islamabad Capital",-,-,1 Kanal,For Rent,2 days ago,A Single Story Building In Which One Hall Plus...
...,...,...,...,...,...,...,...,...,...,...,...
280040,https://www.zameen.com//Property/balochistan_z...,Commercial Plot On National Highway Quetta Roa...,Commercial Plot,PKR3.4 Crore,"Zhob, Balochistan",-,-,19.1 Marla,For Sale,6 years ago,Commercial Plot On National Highway Quetta Roa...
280041,https://www.zameen.com//Property/balochistan_z...,Residential Plot For Sale,Residential Plot,PKR50 Lakh,"Zhob, Balochistan",-,-,4.4 Marla,For Sale,6 years ago,Zhob bypassIdeal locationNear to Pak China Eco...
280042,https://www.zameen.com//Property/balochistan_z...,Residential Plot For Sale,Residential Plot,PKR50 Lakh,"Zhob, Balochistan",-,-,4.4 Marla,For Sale,6 years ago,Zhob bypassIdeal locationNear to Pak China Eco...
280043,https://www.zameen.com//Property/balochistan_z...,Residential Plot For Sale,Residential Plot,PKR50 Lakh,"Zhob, Balochistan",-,-,4.4 Marla,For Sale,6 years ago,Zhob bypassIdeal locationNear to Pak China Eco...


In [ ]:
# Function to calculate number of zeros in a price 
def get_zeroes(price_unit):
    unit = price_unit.strip().lower()
    if unit == 'hundred':
        return 10**2
    elif unit == 'thousand':
        return 10**3
    elif unit == 'lakh':
        return 10**5
    elif unit == 'crore':
        return 10**7
    elif unit == 'arab':
        return 10**9
    elif unit == 'kharab':
        return 10**11
    else:
        return 1      #For price lists like ['PKR1']

# Function to separate price from currency and convert it into float
def get_price_value(row):
    price = row['Price'].split()
    try:
        priceInt = float(price[0].replace(row["Currency"], "")) * get_zeroes(price[-1])
    except:
        priceInt = 0
    return priceInt

# Function to get currency in which the price is mentioned
def get_currency(row):
    price = row['Price'].split()
    match = re.search(r"\d", price[0])   # Locate the digit in the price string
    if match:
        # Get the substring before the first digit
        currency = price[0][:match.start()]
    else:
        currency = None
    return currency

In [ ]:
# Insert new columns for Currency and Price
df.insert(df.columns.get_loc('Price') + 1, 'Currency', df.apply(get_currency, axis = 1))
df.insert(df.columns.get_loc('Price') + 1, 'Price_Value', df.apply(get_price_value, axis=1))
df.head()

,Url,Title,Type,Price,Price_Value,Currency,Location,Baths,Beds,Area,Purpose,Date_Added,Description
0,https://www.zameen.com//Property/islamabad_tar...,10 Kanal Empty Land Available For Rent,Commercial Plot,4 Lakh,400000.0,,"Tarnol, Islamabad, Islamabad Capital",-,-,10 Kanal,For Rent,3 days ago,10 kanal empty land available for rent in Isla...
1,https://www.zameen.com//Property/islamabad_gt_...,Fair-priced 27000 Square Feet Commercial Plot ...,Commercial Plot,15 Lakh,1500000.0,,"GT Road, Islamabad, Islamabad Capital",-,-,6 Kanal,For Rent,2 weeks ago,6 Kanal Land On main GT Road For Rent Ideal Pl...
2,https://www.zameen.com//Property/islamabad_isl...,20 Kanal Land On Main Isb Highway Available Fo...,Commercial Plot,20 Lakh,2000000.0,,"Islamabad Expressway, Islamabad, Islamabad Cap...",-,-,20 Kanal,For Rent,2 weeks ago,20 Kanal Land on Main ISB Highway Available Fo...
3,https://www.zameen.com//Property/islamabad_g_1...,Industrial Building Available For Rent,Industrial Land,55 Thousand,55000.0,,"G-12, Islamabad, Islamabad Capital",-,-,1 Kanal,For Rent,2 days ago,A Industrial Plot Located In G12 Just 1km Away...
4,https://www.zameen.com//Property/islamabad_mir...,Industrial Building Available For Rent,Industrial Land,50 Thousand,50000.0,,"Mira Abadi, Islamabad, Islamabad Capital",-,-,1 Kanal,For Rent,2 days ago,A Single Story Building In Which One Hall Plus...


In [ ]:
# Function to extract the city from location
def get_city(row):
    location = row['Location'].split(',')
    return location[-2].strip()
    
# Function to extract the province from location
def get_province(row):
    location = row['Location'].split(',')
    return location[-1].strip()

In [ ]:
# Insert new columns for City and Province
df.insert(df.columns.get_loc('Location') + 1, 'City', df.apply(get_city, axis=1))
df.insert(df.columns.get_loc('Location') + 1, 'Province', df.apply(get_province, axis = 1))
df.head()

,Url,Title,Type,Price,Price_Value,Currency,Location,Province,City,Baths,Beds,Area,Purpose,Date_Added,Description
0,https://www.zameen.com//Property/islamabad_tar...,10 Kanal Empty Land Available For Rent,Commercial Plot,4 Lakh,400000.0,,"Tarnol, Islamabad, Islamabad Capital",Islamabad Capital,Islamabad,-,-,10 Kanal,For Rent,3 days ago,10 kanal empty land available for rent in Isla...
1,https://www.zameen.com//Property/islamabad_gt_...,Fair-priced 27000 Square Feet Commercial Plot ...,Commercial Plot,15 Lakh,1500000.0,,"GT Road, Islamabad, Islamabad Capital",Islamabad Capital,Islamabad,-,-,6 Kanal,For Rent,2 weeks ago,6 Kanal Land On main GT Road For Rent Ideal Pl...
2,https://www.zameen.com//Property/islamabad_isl...,20 Kanal Land On Main Isb Highway Available Fo...,Commercial Plot,20 Lakh,2000000.0,,"Islamabad Expressway, Islamabad, Islamabad Cap...",Islamabad Capital,Islamabad,-,-,20 Kanal,For Rent,2 weeks ago,20 Kanal Land on Main ISB Highway Available Fo...
3,https://www.zameen.com//Property/islamabad_g_1...,Industrial Building Available For Rent,Industrial Land,55 Thousand,55000.0,,"G-12, Islamabad, Islamabad Capital",Islamabad Capital,Islamabad,-,-,1 Kanal,For Rent,2 days ago,A Industrial Plot Located In G12 Just 1km Away...
4,https://www.zameen.com//Property/islamabad_mir...,Industrial Building Available For Rent,Industrial Land,50 Thousand,50000.0,,"Mira Abadi, Islamabad, Islamabad Capital",Islamabad Capital,Islamabad,-,-,1 Kanal,For Rent,2 days ago,A Single Story Building In Which One Hall Plus...


In [ ]:
# Function to get the number of bathrooms
def get_baths(row):
    match = re.match(r"\d+", row['Baths'])
    return int(row['Baths'][:match.end()]) if match else 0

# Function to get the number of bedrooms
def get_beds(row):
    match = re.match(r"\d+", row['Beds'])
    return int(row['Beds'][:match.end()]) if match else 0

In [ ]:
# Process the Baths and Beds columns to get integer values
df['Baths'] = df.apply(get_baths, axis = 1)
df['Beds'] = df.apply(get_beds, axis = 1)

df.head()

,Url,Title,Type,Price,Price_Value,Currency,Location,Province,City,Baths,Beds,Area,Purpose,Date_Added,Description
0,https://www.zameen.com//Property/islamabad_tar...,10 Kanal Empty Land Available For Rent,Commercial Plot,4 Lakh,400000.0,,"Tarnol, Islamabad, Islamabad Capital",Islamabad Capital,Islamabad,0,0,10 Kanal,For Rent,3 days ago,10 kanal empty land available for rent in Isla...
1,https://www.zameen.com//Property/islamabad_gt_...,Fair-priced 27000 Square Feet Commercial Plot ...,Commercial Plot,15 Lakh,1500000.0,,"GT Road, Islamabad, Islamabad Capital",Islamabad Capital,Islamabad,0,0,6 Kanal,For Rent,2 weeks ago,6 Kanal Land On main GT Road For Rent Ideal Pl...
2,https://www.zameen.com//Property/islamabad_isl...,20 Kanal Land On Main Isb Highway Available Fo...,Commercial Plot,20 Lakh,2000000.0,,"Islamabad Expressway, Islamabad, Islamabad Cap...",Islamabad Capital,Islamabad,0,0,20 Kanal,For Rent,2 weeks ago,20 Kanal Land on Main ISB Highway Available Fo...
3,https://www.zameen.com//Property/islamabad_g_1...,Industrial Building Available For Rent,Industrial Land,55 Thousand,55000.0,,"G-12, Islamabad, Islamabad Capital",Islamabad Capital,Islamabad,0,0,1 Kanal,For Rent,2 days ago,A Industrial Plot Located In G12 Just 1km Away...
4,https://www.zameen.com//Property/islamabad_mir...,Industrial Building Available For Rent,Industrial Land,50 Thousand,50000.0,,"Mira Abadi, Islamabad, Islamabad Capital",Islamabad Capital,Islamabad,0,0,1 Kanal,For Rent,2 days ago,A Single Story Building In Which One Hall Plus...


In [ ]:
# Function to get the conversion factor to convert an area unit into sqft
def conversion_factor(unit):
    unit = unit.strip().lower()
    if unit == "sq. yd.":
        return 9
    elif unit == 'marla':
        return 272.251
    elif unit == 'kanal':
        return 5445
    else:
        return 1

# Function to convert the area into sqft
def get_area_sqft(row):
    match = re.match(r"\d+,\d+", row['Area'])   # Get Area strings of the form 45,000
    if match:
      return float(row['Area'][:match.end()].replace(',', '')) * conversion_factor(row['Area'][match.end():]) 
    else:
      match = re.match(r"\d+(?:\.\d+)?", row['Area'])   # Get Area strings of the form 45 or 4.5
      return float(row['Area'][:match.end()]) * conversion_factor(row['Area'][match.end():]) if match else 0


In [ ]:
# Insert a new column for Area in Square Feet
df.insert(df.columns.get_loc('Area') + 1, 'Area_in_sqft', df.apply(get_area_sqft, axis = 1))
df.head()

,Url,Title,Type,Price,Price_Value,Currency,Location,Province,City,Baths,Beds,Area,Area_in_sqft,Purpose,Date_Added,Description
0,https://www.zameen.com//Property/islamabad_tar...,10 Kanal Empty Land Available For Rent,Commercial Plot,4 Lakh,400000.0,,"Tarnol, Islamabad, Islamabad Capital",Islamabad Capital,Islamabad,0,0,10 Kanal,54450.0,For Rent,3 days ago,10 kanal empty land available for rent in Isla...
1,https://www.zameen.com//Property/islamabad_gt_...,Fair-priced 27000 Square Feet Commercial Plot ...,Commercial Plot,15 Lakh,1500000.0,,"GT Road, Islamabad, Islamabad Capital",Islamabad Capital,Islamabad,0,0,6 Kanal,32670.0,For Rent,2 weeks ago,6 Kanal Land On main GT Road For Rent Ideal Pl...
2,https://www.zameen.com//Property/islamabad_isl...,20 Kanal Land On Main Isb Highway Available Fo...,Commercial Plot,20 Lakh,2000000.0,,"Islamabad Expressway, Islamabad, Islamabad Cap...",Islamabad Capital,Islamabad,0,0,20 Kanal,108900.0,For Rent,2 weeks ago,20 Kanal Land on Main ISB Highway Available Fo...
3,https://www.zameen.com//Property/islamabad_g_1...,Industrial Building Available For Rent,Industrial Land,55 Thousand,55000.0,,"G-12, Islamabad, Islamabad Capital",Islamabad Capital,Islamabad,0,0,1 Kanal,5445.0,For Rent,2 days ago,A Industrial Plot Located In G12 Just 1km Away...
4,https://www.zameen.com//Property/islamabad_mir...,Industrial Building Available For Rent,Industrial Land,50 Thousand,50000.0,,"Mira Abadi, Islamabad, Islamabad Capital",Islamabad Capital,Islamabad,0,0,1 Kanal,5445.0,For Rent,2 days ago,A Single Story Building In Which One Hall Plus...


In [ ]:
# Function to convert string into Date
def convert_date(row):
  # Split the date string into two parts: the number of years/months and the unit (years or months)
  parts = row['Date_Added'].split()
  num = int(parts[0])  # number of years/months
  unit = parts[1]  # unit (years or months)

  # Calculate the number of days based on the unit
  if unit == "years" or unit == "year":
    num_days = num * 365
  elif unit == "months" or unit == "month":
    num_days = num * 30
  elif unit == "weeks" or unit == "week":
    num_days = num * 7
  elif unit == "days" or unit == "day":
    num_days = num
  elif unit == "hours" or unit == "hour":
    num_days = 0
  elif unit == "minutes" or unit == "minute":
    num_days = 0
  elif unit == "seconds" or unit == "second":
    num_days = 0
  else:
    return 0
  
  # Calculate the specific date in the past by subtracting the number of days from the current date
  specific_date = datetime.now() - timedelta(days=num_days)
  specific_date = specific_date.strftime("%Y-%m-%d")

  return specific_date

# Get month from date
def get_month(row):
  return pd.to_datetime(row['Date_Added']).strftime("%m")

# Get year from date
def get_year(row):
  return pd.to_datetime(row['Date_Added']).strftime("%Y")

In [ ]:
# Update the Date_Added column with the estimated dates
df['Date_Added'] = df.apply(convert_date, axis = 1)

#Insert new columns for Month and Year
df.insert(df.columns.get_loc('Date_Added') + 1, 'Month', df.apply(get_month, axis = 1))
df.insert(df.columns.get_loc('Date_Added') + 2, 'Year', df.apply(get_year, axis = 1))

df.head()

,Url,Title,Type,Price,Price_Value,Currency,Location,Province,City,Baths,Beds,Area,Area_in_sqft,Purpose,Date_Added,Month,Year,Description
0,https://www.zameen.com//Property/islamabad_tar...,10 Kanal Empty Land Available For Rent,Commercial Plot,4 Lakh,400000.0,,"Tarnol, Islamabad, Islamabad Capital",Islamabad Capital,Islamabad,0,0,10 Kanal,54450.0,For Rent,2022-12-25,12,2022,10 kanal empty land available for rent in Isla...
1,https://www.zameen.com//Property/islamabad_gt_...,Fair-priced 27000 Square Feet Commercial Plot ...,Commercial Plot,15 Lakh,1500000.0,,"GT Road, Islamabad, Islamabad Capital",Islamabad Capital,Islamabad,0,0,6 Kanal,32670.0,For Rent,2022-12-14,12,2022,6 Kanal Land On main GT Road For Rent Ideal Pl...
2,https://www.zameen.com//Property/islamabad_isl...,20 Kanal Land On Main Isb Highway Available Fo...,Commercial Plot,20 Lakh,2000000.0,,"Islamabad Expressway, Islamabad, Islamabad Cap...",Islamabad Capital,Islamabad,0,0,20 Kanal,108900.0,For Rent,2022-12-14,12,2022,20 Kanal Land on Main ISB Highway Available Fo...
3,https://www.zameen.com//Property/islamabad_g_1...,Industrial Building Available For Rent,Industrial Land,55 Thousand,55000.0,,"G-12, Islamabad, Islamabad Capital",Islamabad Capital,Islamabad,0,0,1 Kanal,5445.0,For Rent,2022-12-26,12,2022,A Industrial Plot Located In G12 Just 1km Away...
4,https://www.zameen.com//Property/islamabad_mir...,Industrial Building Available For Rent,Industrial Land,50 Thousand,50000.0,,"Mira Abadi, Islamabad, Islamabad Capital",Islamabad Capital,Islamabad,0,0,1 Kanal,5445.0,For Rent,2022-12-26,12,2022,A Single Story Building In Which One Hall Plus...


In [ ]:
# Set the Data types of the DataFrame columns
df = df.astype({
    'Url': str,
    'Title': str, 
    'Type': str, 
    'Price': str, 
    'Price_Value': int,
    'Currency': str, 
    'Location': str,
    'Province': str,
    'City': str, 
    'Baths': int, 
    'Beds': int,
    'Area': str, 
    'Area_in_sqft': float,
    'Purpose': str,
    'Date_Added': str,
    'Month': int,
    'Year': int,
    'Description': str
})

In [ ]:
# Get the count for each property type
df["Type"].value_counts()

Residential Plot     115018
House                 72678
Flat                  28091
Commercial Plot       12355
Upper Portion          9980
Shop                   9447
Plot File              9203
Office                 7004
Lower Portion          5828
Building               4176
Agricultural Land      1393
Warehouse               973
Industrial Land         892
Factory                 884
Farm House              706
Room                    657
Other                   388
Penthouse               324
Plot Form                45
Home                      2
Commercial                1
Name: Type, dtype: int64

In [ ]:
# Change the type of Home to House
df.at[df[df["Type"] == "Home"].index, "Type"] = "House"

# Change the type of Commercial to Shop
df.at[df[df["Type"] == "Commercial"].index, "Type"] = "Shop"

# Change the type of Plot Form to Plot File
df.at[df[df["Type"] == "Plot Form"].index, "Type"] = "Plot File"

In [ ]:
# Remove all rows with area = 0
df = df[df['Area_in_sqft'] != 0]

# For all the rows with no currency, set the currency as PKR
noCurrency = df[df['Currency'] == '']
df.loc[noCurrency.index, 'Currency'] = 'PKR'

#Remove Prices Less than 1000
df = df[df["Price_Value"] > 1000]


In [ ]:
# Removing the Area Outliers for each type of property (Threshold = 3)

# Loop through all the property types
for p_type in df['Type'].unique():
  
  # Get a copy of the dataframe with all the properties of a specific type
  p_type_df = df[df['Type'] == p_type]

  # Calculate mean and standard deviation
  area_mean = p_type_df['Area_in_sqft'].mean()
  area_std = p_type_df['Area_in_sqft'].std()

  # Identify and Remove Outliers
  area_outliers = p_type_df[(p_type_df['Area_in_sqft'] < area_mean - 3 * area_std) | (p_type_df['Area_in_sqft'] > area_mean + 3 * area_std)]
  df = df.drop(area_outliers.index)

df
  

,Url,Title,Type,Price,Price_Value,Currency,Location,Province,City,Baths,Beds,Area,Area_in_sqft,Purpose,Date_Added,Month,Year,Description
0,https://www.zameen.com//Property/islamabad_tar...,10 Kanal Empty Land Available For Rent,Commercial Plot,4 Lakh,400000,PKR,"Tarnol, Islamabad, Islamabad Capital",Islamabad Capital,Islamabad,0,0,10 Kanal,54450.0000,For Rent,2022-12-25,12,2022,10 kanal empty land available for rent in Isla...
1,https://www.zameen.com//Property/islamabad_gt_...,Fair-priced 27000 Square Feet Commercial Plot ...,Commercial Plot,15 Lakh,1500000,PKR,"GT Road, Islamabad, Islamabad Capital",Islamabad Capital,Islamabad,0,0,6 Kanal,32670.0000,For Rent,2022-12-14,12,2022,6 Kanal Land On main GT Road For Rent Ideal Pl...
2,https://www.zameen.com//Property/islamabad_isl...,20 Kanal Land On Main Isb Highway Available Fo...,Commercial Plot,20 Lakh,2000000,PKR,"Islamabad Expressway, Islamabad, Islamabad Cap...",Islamabad Capital,Islamabad,0,0,20 Kanal,108900.0000,For Rent,2022-12-14,12,2022,20 Kanal Land on Main ISB Highway Available Fo...
3,https://www.zameen.com//Property/islamabad_g_1...,Industrial Building Available For Rent,Industrial Land,55 Thousand,55000,PKR,"G-12, Islamabad, Islamabad Capital",Islamabad Capital,Islamabad,0,0,1 Kanal,5445.0000,For Rent,2022-12-26,12,2022,A Industrial Plot Located In G12 Just 1km Away...
4,https://www.zameen.com//Property/islamabad_mir...,Industrial Building Available For Rent,Industrial Land,50 Thousand,50000,PKR,"Mira Abadi, Islamabad, Islamabad Capital",Islamabad Capital,Islamabad,0,0,1 Kanal,5445.0000,For Rent,2022-12-26,12,2022,A Single Story Building In Which One Hall Plus...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280040,https://www.zameen.com//Property/balochistan_z...,Commercial Plot On National Highway Quetta Roa...,Commercial Plot,PKR3.4 Crore,34000000,PKR,"Zhob, Balochistan",Balochistan,Zhob,0,0,19.1 Marla,5199.9941,For Sale,2016-12-29,12,2016,Commercial Plot On National Highway Quetta Roa...
280041,https://www.zameen.com//Property/balochistan_z...,Residential Plot For Sale,Residential Plot,PKR50 Lakh,5000000,PKR,"Zhob, Balochistan",Balochistan,Zhob,0,0,4.4 Marla,1197.9044,For Sale,2016-12-29,12,2016,Zhob bypassIdeal locationNear to Pak China Eco...
280042,https://www.zameen.com//Property/balochistan_z...,Residential Plot For Sale,Residential Plot,PKR50 Lakh,5000000,PKR,"Zhob, Balochistan",Balochistan,Zhob,0,0,4.4 Marla,1197.9044,For Sale,2016-12-29,12,2016,Zhob bypassIdeal locationNear to Pak China Eco...
280043,https://www.zameen.com//Property/balochistan_z...,Residential Plot For Sale,Residential Plot,PKR50 Lakh,5000000,PKR,"Zhob, Balochistan",Balochistan,Zhob,0,0,4.4 Marla,1197.9044,For Sale,2016-12-29,12,2016,Zhob bypassIdeal locationNear to Pak China Eco...


In [ ]:
#Removing the Price Outliers for each property type, purpoes and City(Threshold = 3)

# Loop through all the property types
for p_type in df['Type'].unique():
  # Loop through all the purposes i.e., sale and rent
  for purpose in df['Purpose'].unique():
    # Loop through all the cities
    for city in df['City'].unique():

      # Get a copy of the dataframe with all the properties of a specific type, purpose and city
      temp_df = df[(df['Type'] == p_type) & (df['Purpose'] == purpose) & (df['City'] == city)]

      #Calculate mean and standard deviation
      price_mean = temp_df['Price_Value'].mean()
      price_std = temp_df['Price_Value'].std()

      # Identify and Remove Outliers
      price_outliers = temp_df[(temp_df['Price_Value'] < price_mean - 3 * price_std) | (temp_df['Price_Value'] > price_mean + 3 * price_std)]
      df = df.drop(price_outliers.index)

df

,Url,Title,Type,Price,Price_Value,Currency,Location,Province,City,Baths,Beds,Area,Area_in_sqft,Purpose,Date_Added,Month,Year,Description
0,https://www.zameen.com//Property/islamabad_tar...,10 Kanal Empty Land Available For Rent,Commercial Plot,4 Lakh,400000,PKR,"Tarnol, Islamabad, Islamabad Capital",Islamabad Capital,Islamabad,0,0,10 Kanal,54450.0000,For Rent,2022-12-25,12,2022,10 kanal empty land available for rent in Isla...
1,https://www.zameen.com//Property/islamabad_gt_...,Fair-priced 27000 Square Feet Commercial Plot ...,Commercial Plot,15 Lakh,1500000,PKR,"GT Road, Islamabad, Islamabad Capital",Islamabad Capital,Islamabad,0,0,6 Kanal,32670.0000,For Rent,2022-12-14,12,2022,6 Kanal Land On main GT Road For Rent Ideal Pl...
2,https://www.zameen.com//Property/islamabad_isl...,20 Kanal Land On Main Isb Highway Available Fo...,Commercial Plot,20 Lakh,2000000,PKR,"Islamabad Expressway, Islamabad, Islamabad Cap...",Islamabad Capital,Islamabad,0,0,20 Kanal,108900.0000,For Rent,2022-12-14,12,2022,20 Kanal Land on Main ISB Highway Available Fo...
3,https://www.zameen.com//Property/islamabad_g_1...,Industrial Building Available For Rent,Industrial Land,55 Thousand,55000,PKR,"G-12, Islamabad, Islamabad Capital",Islamabad Capital,Islamabad,0,0,1 Kanal,5445.0000,For Rent,2022-12-26,12,2022,A Industrial Plot Located In G12 Just 1km Away...
4,https://www.zameen.com//Property/islamabad_mir...,Industrial Building Available For Rent,Industrial Land,50 Thousand,50000,PKR,"Mira Abadi, Islamabad, Islamabad Capital",Islamabad Capital,Islamabad,0,0,1 Kanal,5445.0000,For Rent,2022-12-26,12,2022,A Single Story Building In Which One Hall Plus...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280040,https://www.zameen.com//Property/balochistan_z...,Commercial Plot On National Highway Quetta Roa...,Commercial Plot,PKR3.4 Crore,34000000,PKR,"Zhob, Balochistan",Balochistan,Zhob,0,0,19.1 Marla,5199.9941,For Sale,2016-12-29,12,2016,Commercial Plot On National Highway Quetta Roa...
280041,https://www.zameen.com//Property/balochistan_z...,Residential Plot For Sale,Residential Plot,PKR50 Lakh,5000000,PKR,"Zhob, Balochistan",Balochistan,Zhob,0,0,4.4 Marla,1197.9044,For Sale,2016-12-29,12,2016,Zhob bypassIdeal locationNear to Pak China Eco...
280042,https://www.zameen.com//Property/balochistan_z...,Residential Plot For Sale,Residential Plot,PKR50 Lakh,5000000,PKR,"Zhob, Balochistan",Balochistan,Zhob,0,0,4.4 Marla,1197.9044,For Sale,2016-12-29,12,2016,Zhob bypassIdeal locationNear to Pak China Eco...
280043,https://www.zameen.com//Property/balochistan_z...,Residential Plot For Sale,Residential Plot,PKR50 Lakh,5000000,PKR,"Zhob, Balochistan",Balochistan,Zhob,0,0,4.4 Marla,1197.9044,For Sale,2016-12-29,12,2016,Zhob bypassIdeal locationNear to Pak China Eco...


In [ ]:
# Convert the pre-processed dataframe into csv file
df.to_csv('/content/drive/MyDrive/DWDM Data/RealEstatePreprocessed.csv', index = False)